In [1]:
#!pip -q install openai

In [28]:
import requests
import http,json
import openai
import os
import pandas as pd
from datetime import datetime
import re
import time

# import local library
import sys
from GPTCall import ask_chatgpt
from SaveExcel_v2 import update_sheet_preserving_format, update_list_sheets_preserving_format

In [3]:
# read the current path, set to mypath
mypath = os.getcwd()
myfile = r"CMG_Processing_MSDManual.xlsx"
process_knowledge_filename = r"CMG_Prompts.xlsx"

# Use os.path.join to create the full paths
process_knowledge_file_fullpath = os.path.join(mypath, process_knowledge_filename.lstrip('/'))
myexcelfile = os.path.join(mypath, myfile.lstrip('/'))

In [4]:
myexcelfile

'c:\\Users\\PC\\NIT6001\\CMG_prog_v3\\CMG_Processing_MSDManual.xlsx'

In [5]:
process_knowledge_file_fullpath

'c:\\Users\\PC\\NIT6001\\CMG_prog_v3\\CMG_Prompts.xlsx'

In [27]:
def keyfunction_readme():
    i=i
    # key function steps
    #-------------------------------------
    # 1.  In mypath folder, there is an excel file called Cognitive Map Graph Processing v3 2024.02.14.xlsx
    # 2.  The excel file has three sheets: articles, paragraphs, and sentences
        # Sheet name: paragraphs
        #Columns: ['ID', 'Paragraph text', 'url', 'category labels', 'summarised key points in simple sentences', 'processing user', 'processing date']

        #Sheet name: sentences
        #Columns: ['ID', 'paragraph ID', 'CMG Auto with GPT', 'CMG by Human Expert', 'Justification of the correction', 'processing user', 'processing date', 'correction user', 'corrction date']

    # 3. Read the articles to a dataframe called df_article, run through it row by row, call ChatGPT API,
    #     if the row processed is not yes, then, ask gpt to group it into major paragraphs and sub pagragphs, add to paragraph df
    #      #Columns: Article ID	Full text	url	category labels	processed	processing user	processing date

    # updated on 7/May 2024
    # 1 read api_key and processing knowledge (promnpt) from an excel file, easier to edit
    # 2. saveExcel to v2, which can save multiple sheets in one call
    # 3. GPTcall allow parameter to change model, tempearture and max_tokens

def cmg_process_1(row_start=0, row_end=0):
    print ("main function started \n--------------------")
    # call the function to read the excel file paragraphs and articles
    df_paragraphs = pd.read_excel(myexcelfile, sheet_name='paragraphs')
    df_articles = pd.read_excel(myexcelfile, sheet_name='articles')

    # set the start and end row to process
    df_paragraphs, df_articles=group_paragraphs(df_paragraphs,  df_articles, row_start, row_end)

    data_sheets = [('paragraphs', df_paragraphs), ('articles', df_articles)]
    update_list_sheets_preserving_format(myexcelfile, data_sheets)


def check_excelfile_info(myexcelfile):
    # Use 'with' to ensure the Excel file is closed after checking
    with pd.ExcelFile(myexcelfile) as xls:
        # Iterate through all sheets
        for sheet_name in xls.sheet_names:
            # Read each sheet
            df = pd.read_excel(xls, sheet_name)

            # Print the sheet name and its columns
            print(f"Sheet name: {sheet_name}")
            print("Columns:", df.columns.tolist())

    # The file is automatically closed when exiting the 'with' block

def generate_prompt(step, prompt_path=process_knowledge_file_fullpath):
    # Map step numbers to knowledge areas
    step_mapping = {
        1: "step1_group_paragraphs",
        2: "step2_summarise_paragraphs",
        3: "step3_convert_sentence_to_cognitive_map_graph"
    }
    
    # Check if the input step is valid
    if step not in step_mapping:
        return "Error: Invalid step input. Please enter 1, 2, or 3."
    
    # Retrieve the corresponding knowledge area
    knowledge_area = step_mapping[step]
    
    # Read the prompts from the excel file, sheet name: knowledge, column: knowledge_area
    df = pd.read_excel(prompt_path, sheet_name="knowledge", engine='openpyxl')
    
    # Filter based on the knowledge_area
    filtered_df = df[df['knowledge_area'] == knowledge_area]
    
    if filtered_df.empty:
        return f"Could not read the knowledge on {knowledge_area}.\n"
    
    myprompt = '\n'.join(filtered_df['knowledge'].astype(str))
    
    # Customize the message based on the step
    if step == 1:
        return myprompt + "\n Here is the article content:"
    elif step == 2:
        return myprompt + "\n Here is the paragraph:"
    elif step == 3:
        return myprompt + "\n Here is the sentence:"

def group_paragraphs(df_paragraphs, df_articles, row_start, row_end):
    print("\n group_paragraphs function \n --------------------------------------")
    
    # Generate the prompt for grouping paragraphs (step 1)
    myprompt = generate_prompt(1)
    print(myprompt)

    # Adjust row_end if it's 0 (process until the last row)
    if row_end == 0 or row_end > df_articles.index[-1]:   
        row_end = df_articles.index[-1]
    
    # Ensure row_start is within bounds
    if row_start < 0:
        row_start = 0
    
    # Ensure row_end is not before row_start
    if row_end < row_start:
        row_end = row_start
    
    # Loop through the articles from row_start to row_end
    for index in range(row_start, row_end + 1):
        if index > df_articles.index[-1]:  # Ensure index is within DataFrame bounds
            break  # Exit the loop if index exceeds the DataFrame row count

        # Access the current row
        row = df_articles.iloc[index]
        article_id = row['Article ID']
        fulltext = str(row['Full text'])
        processed_flag = row['processed']
        
        # Check if the article has already been processed and the fulltext is valid
        if processed_flag != 'Yes' and fulltext and fulltext.lower() != 'nan':
            # Call the GPT model with the prompt and full text
            response_text = ask_chatgpt(myprompt, fulltext)
            print("-------response_text-----------------------")
            print(response_text)

            # Parse the response and update df_paragraphs and df_articles
            df_paragraphs, article_label, article_metadata, article_keypoint = parse_paragraphs_json_v2(response_text, article_id, df_paragraphs)
            print("article_metadata: ", article_metadata) # 666
            # Mark the article as processed and store relevant data
            df_articles.loc[index, 'processed'] = 'Yes'
            df_articles.loc[index, 'article labels'] = ', '.join(article_label) if article_label else None
            df_articles.loc[index, 'article metadata'] = json.dumps(article_metadata)
            df_articles.loc[index, 'article keypoint'] = ', '.join(article_keypoint) if article_keypoint else None
            df_articles.loc[index, 'json str'] = response_text

    return df_paragraphs, df_articles


def parse_paragraphs_json_v2(response_text, article_id, df_paragraphs):
    # Load the response text as JSON
    try:
        response_json = response_text.replace("```json", "").replace("```", "") # Remove code block markdown
        response_json = json.loads(response_json)
    except json.JSONDecodeError:
        print("Error decoding JSON")
        return df_paragraphs, [], [], []
    
    # Extract relevant parts from the JSON
    article_label = response_json.get("article_label", [])
    article_metadata = response_json.get("article_metadata", [])
    print("article_metadata: ", article_metadata) # 666
    article_keypoint = response_json.get("article_keypoint", [])
    paragraphs = response_json.get("paragraphs", [])

    # Get the last ID from the DataFrame or start from 0 if empty
    last_id = df_paragraphs['ID'].max() if not df_paragraphs.empty else 0
    
    # Prepare a list to hold new rows for batch concatenation
    new_rows = []

    # Process each paragraph
    for paragraph in paragraphs:
        if isinstance(paragraph, dict):
            # Extract label and paragraph text from the dictionary
            label = paragraph.get("label", [])
            new_paragraph = paragraph.get("new_paragraph", "")
        else:
            # Handle unexpected data formats
            label = ["Unexpected data format error"]
            new_paragraph = str(paragraph)

        # Increment the ID and prepare a new row for the DataFrame
        last_id += 1
        new_row = {
            'ID': last_id,
            'Article ID': article_id,
            'paragraph labels': ', '.join(label),  # Convert label list to comma-separated string
            'Paragraph text': new_paragraph,
        }
        new_rows.append(new_row)
    
    # Concatenate the new rows to the existing DataFrame in a single operation
    if new_rows:
        df_paragraphs = pd.concat([df_paragraphs, pd.DataFrame(new_rows)], ignore_index=True)

    # Return the updated DataFrame, article labels, metadata, and key points
    return df_paragraphs, article_label, article_metadata, article_keypoint


In [26]:
cmg_process_1(0, 1)

main function started 
--------------------

 group_paragraphs function 
 --------------------------------------
Let's work this out in a step by step way to be sure we have the right answer.
<instruction>
Coherently split/breakdown this document into meaningful paragraphs. Keep metadata into a separated part.
A document have a list of labels that: summarize the key areas covered in the article.
Each paragraph can have a few labels. Labels represent the category and key info of the article.
A document has Key points. Keep Key points into a separated part.
</instruction>
<format>
Paragraphs should maintain natural language form.
All labels are in a list. 
Labels should have full meaning, because each paragraph will be processed independently. For example, clinic observation is not a good label; 'Acute kidney injury clinic observation' is a clear label. 
Give json as the output, in this structure {"article_label": [***,***], "article_metadata": {metadata of the article}, "article_keypoin

C:\Users\PC\AppData\Local\Temp\ipykernel_29348\2892682442.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Yes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_articles.loc[index, 'processed'] = 'Yes'
C:\Users\PC\AppData\Local\Temp\ipykernel_29348\2892682442.py:125: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Gastrointestinal Endoscopy, Diagnostic Procedures, Therapeutic Procedures, Contraindications, Preparation, Video Capsule Endoscopy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_articles.loc[index, 'article labels'] = ', '.join(article_label) if article_label else None
C:\Users\PC\AppData\Local\Temp\ipykernel_29348\2892682442.py:127: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an e

['sk-proj-pjGUyMNiHPDiH7CLYv4rT3BlbkFJQpZiY1U0sYAqOATygpnJ', 'org-UwdtTWVA8m7W2PxkJfK0Y6Bk', 'proj_fskMYW8tFnU2N8vV9fOaQooF']
{'id': 'chatcmpl-A8qS8R2uOllg4e8IXrmHDw2dxUA0r', 'object': 'chat.completion', 'created': 1726671616, 'model': 'gpt-4o-2024-08-06', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n    "article_label": [\n        "Colonic Diverticulosis Overview",\n        "Symptoms and Diagnosis",\n        "Treatment and Management",\n        "Complications and Risks",\n        "Symptomatic Uncomplicated Diverticular Disease"\n    ],\n    "article_metadata": {\n        "authors": [\n            "Joel A. Baum, MD",\n            "Rafael Antonio Ching Companioni, MD"\n        ],\n        "affiliations": [\n            "Icahn School of Medicine at Mount Sinai",\n            "HCA Florida Gulf Coast Hospital"\n        ],\n        "review_date": "Oct 2022"\n    },\n    "article_keypoint": [\n        "Colonic diverticula are saclike mucosal pouches that 

In [ ]:
def cmg_process_2(row_start=0, row_end=0):
    print ("main function started \n--------------------")
    time_started=time.time()

    df_paragraphs = pd.read_excel(myexcelfile, sheet_name='paragraphs')
    df_sentences = pd.read_excel(myexcelfile, sheet_name='sentences')

    df_paragraphs, df_sentences=extract_events(df_paragraphs,  df_sentences, row_start, row_end)

    update_sheet_preserving_format(myexcelfile, 'paragraphs', df_paragraphs)
    update_sheet_preserving_format(myexcelfile, 'sentences', df_sentences)

    time_finished=time.time()
    timeused=time_finished-time_started
    print("Time used=", round(timeused,2)) # 


def extract_events(df_paragraphs, df_sentences,row_start, row_end):
    print("\n extract_events function \n --------------------------------------")
    myprompt=generate_prompt(2)
    print(myprompt)

    # Adjust row_end if it's 0 (process until the last row)
    if row_end == 0 or row_end > df_paragraphs.index[-1]:   
        row_end = df_paragraphs.index[-1]
    
    # Ensure row_start is within bounds
    if row_start < 0:
        row_start = 0
    
    # Ensure row_end is not before row_start
    if row_end < row_start:
        row_end = row_start
    
    # Loop through the articles from row_start to row_end
    for index in range(row_start, row_end + 1):
        if index > df_paragraphs.index[-1]:  # Ensure index is within DataFrame bounds
            break  # Exit the loop if index exceeds the DataFrame row count

        # Access the current row
        row = df_paragraphs.iloc[index]
        mycontent = row['Paragraph text']
        events_text = row['Convert paragraph to events']
        processedflag=row['processed']

        # Proceed if 'Paragraph text' is not empty and processed is not 'Yes'
        if processedflag!='Yes' and pd.notna(mycontent)  and mycontent.strip():
            response_text = ask_chatgpt(myprompt, mycontent)
            print("-------response_text-----------------------")
            print(response_text)

            # Update the DataFrame with the response
            df_paragraphs.at[index, 'Convert paragraph to events'] = response_text
            paragraph=response_text
            df_sentences=write_summarisedPoints_to_sentence_rows(paragraph,index, df_sentences)
            df_paragraphs.at[index,'processed']='Yes'

    return df_paragraphs,df_sentences


def write_summarisedPoints_to_sentence_rows(paragraph,index, df_sentences):
    print ("\n split_sentences function index=",index," \n -------------------------------------")
    print(type(df_sentences))
    print(df_sentences)
    if not df_sentences.empty:
        # If df_sentences is not empty, continue IDs from the last used ID
        sentence_id = df_sentences['Sentence ID'].max() + 1
    else:  sentence_id = 1  # If df_sentences is empty, start IDs from 1

    new_rows = []  # Initialize a list to hold new rows
    paragraph_id = index+1
    # summarised_sentences = paragraph.split('\n')

    # for sentence in summarised_sentences:
    #         #print("\n here=", sentence)
    #         if sentence and sentence.strip():  # Check if the sentence is not just whitespace
    #             # Create a new row with existing columns, setting default values for unspecified columns
    #             new_row = {col: '' for col in df_sentences.columns}  # Initialize all columns to default values
    #             new_row.update({
    #                 'Sentence ID': sentence_id,
    #                 'Paragraph ID': paragraph_id,
    #                 'Sentence text': sentence.strip()
    #             })
    #             new_rows.append(new_row)
    #             sentence_id += 1

    new_row = {col: '' for col in df_sentences.columns}
    new_row.update({
        'Sentence ID': sentence_id,
        'Paragraph ID': paragraph_id,
        'Sentence text': paragraph
    })
    new_rows.append(new_row)
    sentence_id += 1
    # Append new rows to df_sentences DataFrame
    if new_rows:
        df_sentences = pd.concat([df_sentences, pd.DataFrame(new_rows)], ignore_index=True)

    return df_sentences